In [7]:
from pydantic import BaseModel
from typing import List, Literal

class Answer(BaseModel):
    response: Literal["Yes", "No"]

In [ ]:
from langchain_ollama import ChatOllama
model = ChatOllama(model='gemma3:1b')
model_struct = model.with_structured_output(Answer)

In [18]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

prompt = """You are given a query / task. If you understand the query / have knowledge about the topic, return a 'Yes'.
If you don't have clarity on the asked topic / query, simply return a 'No'.
"""

model_struct.invoke([SystemMessage(content=prompt), HumanMessage(content = "Do you know about kj2")])

Answer(response='Yes')

In [ ]:
from langgraph.graph import MessagesState

class TempState(MessagesState):
    temp: str

In [28]:
def node1(state):
    answer = model_struct.invoke(state['messages'])
    return ({'messages': AIMessage(content=f"{answer.response}")})

In [29]:
from langgraph.graph import MessagesState, START, END, StateGraph

builder = StateGraph(TempState)
builder.add_node('node1', node1)

builder.add_edge(START, "node1")
builder.add_edge("node1", END)

graph = builder.compile()

In [30]:
graph.invoke({'messages':'Hey do you know about Ks2'})

{'messages': [HumanMessage(content='Hey do you know about Ks2', additional_kwargs={}, response_metadata={}, id='d7cb0be0-4231-49b4-b765-d1334abe7506'),
  AIMessage(content='Yes', additional_kwargs={}, response_metadata={}, id='801589d1-9140-4d4d-94ed-5173fa6b9754')]}

In [31]:
from langgraph.graph import MessagesState
from typing import List, Literal, TypedDict

class State(MessagesState):
    answers: List[Literal["Yes", "No"]]

In [35]:
def main_node(state):
    ans = model_struct.invoke(state['messages'])
    return {'answers': ans.response}

def conditional(state) -> Literal['__end__', "fallback_node"]:
    if state['answers'][-1] == "Yes":
        return "__end__"
    elif state['answers'][-1] == "No":
        return "fallback_node"

def fallback_node(state):
    prompt = f"""I am sorry I dont understand your question "{state['messages'][-1]}", Could you please rephrase it, or maybe tell me what it means?"""
    
    return ({'messages':AIMessage(content=prompt)})

In [37]:
builder = StateGraph(State)

builder.add_node("main_node", main_node)
builder.add_node("fallback_node", fallback_node)

builder.add_edge(START, "main_node")
builder.add_conditional_edges("main_node", conditional)
builder.add_edge("fallback_node", END)

graph = builder.compile()

In [39]:
graph.invoke({'messages':"What is khkjh?"})

KeyError: None